In [1]:
from pymongo import MongoClient
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re

client = MongoClient("localhost", 27017)
randomTweetsColl = client["idea"]["regularUser_en_fixed"]
BDPTweetsColl = client["idea"]["BDP_tweets"]
GeoTweetsColl = client["idea"]["geo_tweets_2"]

In [2]:


def load_stopwords(file_location="SmartStoplist"):
    f = open(file_location)
    stopwords = [line.strip() for line in f]
    return stopwords + ["http","https", "don", "thi","http ", "co","dont","im"]

stopwords = load_stopwords()
stemmer = PorterStemmer()
stopSet = set(stopwords)

def preprocessor(text, stopSet):
    stemmer = PorterStemmer()
    words = []
    
    for word in text.lower().split():
        word = re.sub('[!@#$?\'\"]|&amp', '',word)
        stemmedWord = stemmer.stem_word(word)
        condition_Word = word not in stopSet  and "http" not in word
        condition_StemmedWord = stemmedWord not in stopSet
        
        if  condition_Word and  condition_StemmedWord:
            words.append(stemmedWord)
    return " ".join(words)
        

In [3]:
randomTweets = ""
debugger = []
for tweet in randomTweetsColl.find():
    text = preprocessor(tweet["text"],stopSet)
    if "amp " in text:
        debugger.append(text)
    randomTweets += (text + "\n")
    
    
BDPTweets = ""
for tweet in BDPTweetsColl.find():
    text = preprocessor(tweet["text"],stopSet)
    BDPTweets += (text + "\n")




In [4]:


tfidf = TfidfVectorizer(stop_words="english",ngram_range = (1,2))
wordScores = tfidf.fit_transform([randomTweets, BDPTweets])
vector2word = tfidf.get_feature_names()

In [5]:
randomVectors = np.argsort(wordScores[0].todense())
BDPVectors = np.argsort(wordScores[1].todense())
#GeoVectors = np.argsort(wordScores[2].todense())


In [6]:
k = 30
randomKeywords = [vector2word[vector] for vector in randomVectors[:,:].tolist()[0]][::-1]
BDPKeywords = [vector2word[vector] for vector in BDPVectors[:,:].tolist()[0]][::-1]
#GeoKeywords = [vector2word[vector] for vector in GeoVectors[:,-k:].tolist()[0]]

In [7]:
BDPKeywords[:k]   #BDP患者Top k keywords

['bpdchat',
 'mhchat',
 'feel',
 'love',
 'bpd',
 'carldunnjr',
 'time',
 'good',
 'day',
 'make',
 'peopl',
 'today',
 'bpdff',
 'work',
 'ive',
 'lt',
 'thing',
 'life',
 'slhxox',
 'hope',
 'year',
 'ill',
 'night',
 'happi',
 'friend',
 'week',
 'person',
 'xx',
 'lol',
 'fuck']

In [8]:
randomKeywords[:k]   #隨機使用者Top K keywords

['love',
 'day',
 'today',
 'good',
 'time',
 'lol',
 'make',
 'work',
 'follow',
 'peopl',
 'feel',
 'life',
 'watch',
 'fuck',
 'great',
 'night',
 'ive',
 'thing',
 'happi',
 'gt',
 'year',
 'video',
 'game',
 'gameinsight',
 'girl',
 'miss',
 'play',
 'man',
 'tonight',
 'start']

In [17]:


tfidf.transform([["awesome", "cool"]])



AttributeError: 'list' object has no attribute 'lower'